# **IBM Applied Data Science Capstone Course by Coursera**

 * *Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.*
 * *Get the geographical coordinates of the neighborhoods in Toronto.*
 * *Explore and cluster the neighborhoods in Toronto (replicate the same analysis we did to New York City data).*
 
 -------
 ## **1. Import Library**


In [1]:
!pip install folium

     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:011


In [2]:
import numpy as np #to handle file in vectorize manner
import pandas as pd #for data analysis
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json #library to handle Json file

from geopy.geocoders import Nominatim # convert an adress into laditude and longitude

import requests #library to handle request

from bs4 import BeautifulSoup #library to parse HTML and XML documnents

from pandas.io.json import json_normalize #transform Json file into a pandas dataframe

#matplotlib and associated plotting module
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k mean from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Library imported.')

Library imported.


## 2. Scrap data from wikipedia page into DataFrame

In [3]:
# send get request
data= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
#parse data from the html into a beautifulsoup object
soup=BeautifulSoup(data,'html.parser')

In [5]:
#create three list to store table data
postalCodeList=[]
boroughList=[]
neighborhoodList=[]

**Using beautifulSoup**

In [6]:
#find the table
soup.find('table').find_all('tr')

#find all rows of the table
soup.find('table').find_all('tr')

#for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells=row.find_all('td')

In [7]:
#append the data into the resoective lists
for row in soup.find('table').find_all('tr'):
    cells=row.find_all('td')
    if(len(cells)>0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n')) #avoid new line in neighbourhood cell

In [8]:
#create a new dataFrame from the three lists

toronto_df=pd.DataFrame({'PostalCode':postalCodeList,'Borough':boroughList,'Neighborhood':neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 3. Drop the cell a borough that is "Not assigned"

In [9]:
#drop cells with borough that is not assigned
toronto_df_dropna=toronto_df[toronto_df.Borough !="Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 4. Group neighborhood in the same borough

In [10]:
#group neighborhood in the same borough
toronto_df_grouped=toronto_df_dropna.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 5. For Neighborhood='Not assigned', make the value the same as Borough

In [11]:
#for neighborhood='Not assigned', make the value the same as Borough

for index, row in toronto_df_grouped.iterrows():
    if row['Neighborhood']=='Not assigned':
        row['Neighborhood']=row['Borough']
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 6. Check whether it is the same as required by the question 

In [12]:
# create a new test dataframe
column_names=['PostalCode','Borough','Neighborhood']
test_df=pd.DataFrame(columns=column_names)

test_list=["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df=test_df.append(toronto_df_grouped[toronto_df_grouped['PostalCode']==postcode],ignore_index=True)

test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


## 7. Print the number of rows of the cleaned dataframe

In [13]:
# print the number of rows of the clean data frame
toronto_df_grouped.shape

(103, 3)

## 8. Load the coordinates from the csv file on Coursera

In [14]:
# Change the data source
# loda coronavirus from the on Coursera
coordinates = pd.read_csv("https://raw.githubusercontent.com/limchiahooi/Coursera_Capstone/master/Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# rename the column "PostalCode"
coordinates.rename(columns={'Postal Code':'PostalCode'},inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 9. Merge two table to get coordinates

In [16]:
# merge two table on the column "postalcode"
toronto_df_new=toronto_df_grouped.merge(coordinates,on="PostalCode",how='left')
toronto_df_new.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 10. finally, check to make sure the coordinator added as required by the question

In [17]:
#create a new test data frame
column_names=["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]

test_df=pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df=test_df.append(toronto_df_new[toronto_df_new['PostalCode']==postcode],ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## 11. Use geopy library to get the latitude and longitude value of the Toronto

In [18]:
address='Toronto'

geolocator=Nominatim(user_agent='My-application')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical cordinate of toronto are {},{}.'.format(latitude,longitude))

The geographical cordinate of toronto are 43.6534817,-79.3839347.


## 12. Create a map of Toronto with neighborhoods superimposed on top

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)

#add marker to map

for lat, lng, borough,neighborhood in zip(toronto_df_new['Latitude'],toronto_df_new['Longitude'],toronto_df_new['Borough'],toronto_df_new['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label= folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_toronto)
    
map_toronto

## 13. Filter only boroughs that contain the word toronto

In [20]:
#filter borough name that contain the word Toronto

borough_name= list(toronto_df_new.Borough.unique())
borough_with_toronto=[]

for x in borough_name:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [21]:
# create a new data frame with only boroughs that contain word Toronto
toronto_df_new=toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
# map of Toronto and latitude and longitude values

map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)

#add Marker to map

for lat,lng,borough,neighborhood in zip(toronto_df_new['Latitude'],toronto_df_new['Longitude'],toronto_df_new['Borough'],toronto_df_new['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label= folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_toronto)

map_toronto

## Use the Foursquare API to explore thr neighborhoods

In [29]:
#define Foursquare Credentials and Version
CLIENT_ID='BMO3Z2X2BA3P4W00553PUMKFNNXCQCBYNWKNJV5QPKTROM5B' #your Foursquare Id
CLIENT_SECRET='OG4BUOB4XLYNBCA1YNNWBYF54YX1UT0ZIEGJP3E1JVHC3CUG' #your Foursquare Secret
VERSION='20200708' # Foursquare API version Date

print('Your credential:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+ CLIENT_SECRET)

Your credential:
CLIENT_ID: BMO3Z2X2BA3P4W00553PUMKFNNXCQCBYNWKNJV5QPKTROM5B
CLIENT_SECRET: OG4BUOB4XLYNBCA1YNNWBYF54YX1UT0ZIEGJP3E1JVHC3CUG


**Now let's get the top 100 venues that are within a radius of 500 meters.**

In [30]:
radius=500
LIMIT=100

venues=[]

for lat,long,post,borough,neighborhood in zip(toronto_df_new['Latitude'],toronto_df_new['Longitude'],toronto_df_new['PostalCode'],toronto_df_new['Borough'],
                                             toronto_df_new['Neighborhood']):
    url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&redius={}&limit={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    long,
    radius,
    LIMIT)
    
    results=requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
        post,
        borough,
        neighborhood,
        lat,
        long,
        venue['venue']['name'],
        venue['venue']['location']['lat'],
        venue['venue']['location']['lng'],
        venue['venue']['categories'][0]['name']))

In [31]:
#convert the venue list into a new DataFrame

venues_df=pd.DataFrame(venues)

#define the columns names
venues_df.columns=['PostalCode','Borough','Neighborhood','BoroughLatitude','BoroughLongitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory']

print(venues_df.shape)
venues_df.head()

(3868, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail


**Let's check how many venues were returned for each PostalCode**

In [32]:
venues_df.groupby(['PostalCode','Borough','Neighborhood']).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,100,100,100,100,100,100
M4K,East Toronto,"The Danforth West, Riverdale",100,100,100,100,100,100
M4L,East Toronto,"India Bazaar, The Beaches West",100,100,100,100,100,100
M4M,East Toronto,Studio District,100,100,100,100,100,100
M4N,Central Toronto,Lawrence Park,100,100,100,100,100,100
M4P,Central Toronto,Davisville North,100,100,100,100,100,100
M4R,Central Toronto,"North Toronto West, Lawrence Park",100,100,100,100,100,100
M4S,Central Toronto,Davisville,100,100,100,100,100,100
M4T,Central Toronto,"Moore Park, Summerhill East",100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [33]:
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 275 unique categories.


In [34]:
venues_df['VenueCategory'].unique()[:50]

array(['Gastropub', 'Indie Movie Theater',
       'Vegetarian / Vegan Restaurant', 'Ice Cream Shop', 'Trail',
       'Bakery', 'Park', 'Bagel Shop', 'Beach', 'Toy / Game Store',
       'Coffee Shop', 'Health Food Store', 'French Restaurant',
       'BBQ Joint', 'Breakfast Spot', 'Middle Eastern Restaurant', 'Pub',
       'Nail Salon', 'Skating Rink', 'Mexican Restaurant', 'Café',
       'Japanese Restaurant', 'Juice Bar', 'Liquor Store', 'Cupcake Shop',
       'Hungarian Restaurant', 'Butcher', 'Burger Joint',
       'Asian Restaurant', 'Diner', 'Tea Room',
       'Mediterranean Restaurant', 'Gym', 'Brewery', 'Indian Restaurant',
       'Caribbean Restaurant', 'Thai Restaurant', 'Playground',
       'Farmers Market', 'Golf Course', 'Soccer Field', 'Bar', 'Dog Run',
       'Ethiopian Restaurant', 'American Restaurant', 'Comic Shop',
       'Pharmacy', 'Fish & Chips Shop', 'Egyptian Restaurant',
       'Grocery Store'], dtype=object)

## 15. Analyze Each Area

In [35]:
# one hot encoding
toronto_onehot=pd.get_dummies(venues_df[['VenueCategory']],prefix="",prefix_sep="")

#add postal, borough and neighborhood column back to dataFrame

toronto_onehot['PostalCode']=venues_df['PostalCode']
toronto_onehot['Borough']=venues_df['Borough']
toronto_onehot['Neighborhoods']=venues_df['Neighborhood']

#move postal, borough and neighborhood column to the first column

fixed_columns=list(toronto_onehot.columns[-3:])+list(toronto_onehot.columns[:-3])
toronto_onehot=toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(3868, 278)


,PostalCode,Borough,Neighborhoods,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [36]:
toronto_grouped=toronto_onehot.groupby(['PostalCode','Borough','Neighborhoods']).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 278)


,PostalCode,Borough,Neighborhoods,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M4E,East Toronto,The Beaches,0.00,0.01,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.01,0.00

**create new dataframe and display the top 10 venue for each postal code**

In [37]:
num_top_venues=10
indicators=['st','nd','rd']

#create column according to number of top veues

areaColumns=['PostalCode','Borough','Neighborhoods']
freqColumns=[]
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1,indicator[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns=areaColumns+freqColumns

#create a new dataframe
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode']=toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough']=toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods']=toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories=toronto_grouped.iloc[ind,:].iloc[3:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    neighborhoods_venues_sorted.iloc[ind,3:]=row_categories_sorted.index.values[0:num_top_venues]
    
    #neighbohoods_venues_sorted.sort_values(freqColumns,inplace=True)
    
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Park,Coffee Shop,Beach,Café,Bakery,Pub,Breakfast Spot,Indian Restaurant,BBQ Joint,Ice Cream Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",Café,Greek Restaurant,Park,Vietnamese Restaurant,Bakery,Pub,Coffee Shop,Italian Restaurant,Ice Cream Shop,Cocktail Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",Coffee Shop,Café,Park,Brewery,Beach,BBQ Joint,Bar,Indian Restaurant,Bakery,Pizza Place
3,M4M,East Toronto,Studio District,Coffee Shop,Park,Café,Vietnamese Restaurant,Brewery,Bar,Bakery,Diner,Pizza Place,French Restaurant
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Italian Restaurant,Park,Café,Sushi Restaurant,Pizza Place,Bookstore,Japanese Restaurant,Bakery,Ice Cream Shop
5,M4P,Central Toronto,Davisville North,Coffee Shop,Italian Restaurant,Bakery,Café,Park,Indian Restaurant,Deli / Bodega,Restaurant,Japanese Restaurant,Bookstore
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Italian Restaurant,Coffee Shop,Bakery,Café,Sushi Restaurant,Japanese Restaurant,Park,Pizza Place,Bookstore,Restaurant
7,M4S,Central Toronto,Davisville,Italian Restaurant,Coffee Shop,Pizza Place,Bakery,Indian Restaurant,Park,Japanese Restaurant,Café,Restaurant,Supermarket
8,M4T,Central Toronto,"Moore Park, Summerhill East",Italian Restaurant,Park,Sushi Restaurant,Café,Coffee Shop,Bakery,Dessert Shop,Restaurant,Indian Restaurant,Spa
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Italian Restaurant,Café,Coffee Shop,Park,Sushi Restaurant,Yoga Studio,Vegetarian / Vegan Restaurant,Spa,Burger Joint,Bakery


## 16. Cluster Area

**Run kmean to cluster the Toronto into 5 clusters**

In [38]:
#set number of clusters

kclusters=5
toronto_grouped_clustering=toronto_grouped.drop(["PostalCode","Borough","Neighborhoods"],1)

#run k mean clustering
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)

#check cluster labels generated for each row in te dataFrame
kmeans.labels_[0:10]

array([0, 2, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [39]:
#create new dataframe the include the cluster as well as top 10 venues for each neighborhood.
toronto_merged=toronto_df_new.copy()

#add clustering labels
toronto_merged["cluster Labels"]=kmeans.labels_

#merge toronto_grouped with toronto_data to add to add latitude/longitud for each neighborhood
toronto_merged=toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough","Neighborhoods"],1).set_index("PostalCode"),on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() #check latest column!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Park,Coffee Shop,Beach,Café,Bakery,Pub,Breakfast Spot,Indian Restaurant,BBQ Joint,Ice Cream Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Café,Greek Restaurant,Park,Vietnamese Restaurant,Bakery,Pub,Coffee Shop,Italian Restaurant,Ice Cream Shop,Cocktail Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Coffee Shop,Café,Park,Brewery,Beach,BBQ Joint,Bar,Indian Restaurant,Bakery,Pizza Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Park,Café,Vietnamese Restaurant,Brewery,Bar,Bakery,Diner,Pizza Place,French Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Coffee Shop,Italian Restaurant,Park,Café,Sushi Restaurant,Pizza Place,Bookstore,Japanese Restaurant,Bakery,Ice Cream Shop


In [40]:
#sort the result by cluster labels
print(toronto_merged.shape)
toronto_merged.sort_values(["cluster Labels"],inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Park,Coffee Shop,Beach,Café,Bakery,Pub,Breakfast Spot,Indian Restaurant,BBQ Joint,Ice Cream Shop
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0,Coffee Shop,Café,Pub,Park,Japanese Restaurant,Bakery,Thai Restaurant,Gastropub,Ice Cream Shop,Breakfast Spot
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Park,Café,Vietnamese Restaurant,Brewery,Bar,Bakery,Diner,Pizza Place,French Restaurant
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0,Park,Coffee Shop,Brewery,Beach,Bakery,Café,Indian Restaurant,Italian Restaurant,Pizza Place,Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Coffee Shop,Café,Park,Brewery,Beach,BBQ Joint,Bar,Indian Restaurant,Bakery,Pizza Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Coffee Shop,Italian Restaurant,Park,Café,Sushi Restaurant,Pizza Place,Bookstore,Japanese Restaurant,Bakery,Ice Cream Shop
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Coffee Shop,Italian Restaurant,Bakery,Café,Park,Indian Restaurant,Deli / Bodega,Restaurant,Japanese Restaurant,Bookstore
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Italian Restaurant,Coffee Shop,Bakery,Café,Sushi Restaurant,Japanese Restaurant,Park,Pizza Place,Bookstore,Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Italian Restaurant,Coffee Shop,Pizza Place,Bakery,Indian Restaurant,Park,Japanese Restaurant,Café,Restaurant,Supermarket
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Italian Restaurant,Park,Sushi Restaurant,Café,Coffee Shop,Bakery,Dessert Shop,Restaurant,Indian Restaurant,Spa


In [41]:
#create map
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)

#set color scheme for the clusters
x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
color_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in color_array]

#add marker to map

markers_colors=[]
for lat,lon,post,bor,poi,cluster in zip(toronto_merged["Latitude"],toronto_merged['Longitude'],toronto_merged['PostalCode'],toronto_merged['Borough'],toronto_merged['Neighborhood'],toronto_merged['cluster Labels']):
     label=folium.Popup('{}({}):{}-cluster {}'.format(bor,post,poi,cluster),parse_html=True)
     folium.CircleMarker(
     [lat,lon],
     radius=5,
     popup=label,
     color=rainbow[cluster-1],
     fill=True,
     fill_color=rainbow[cluster-1],
     fill_opacity=0.7).add_to(map_clusters)
        
map_clusters

## 17.Examine Clusters

In [44]:
#cluster 1
toronto_merged.loc[toronto_merged["cluster Labels"]==0,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Park,Coffee Shop,Beach,Café,Bakery,Pub,Breakfast Spot,Indian Restaurant,BBQ Joint,Ice Cream Shop
11,Downtown Toronto,0,Coffee Shop,Café,Pub,Park,Japanese Restaurant,Bakery,Thai Restaurant,Gastropub,Ice Cream Shop,Breakfast Spot
3,East Toronto,0,Coffee Shop,Park,Café,Vietnamese Restaurant,Brewery,Bar,Bakery,Diner,Pizza Place,French Restaurant
38,East Toronto,0,Park,Coffee Shop,Brewery,Beach,Bakery,Café,Indian Restaurant,Italian Restaurant,Pizza Place,Bar
2,East Toronto,0,Coffee Shop,Café,Park,Brewery,Beach,BBQ Joint,Bar,Indian Restaurant,Bakery,Pizza Place


In [45]:
#cluster 2
toronto_merged.loc[toronto_merged["cluster Labels"]==1,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,1,Coffee Shop,Italian Restaurant,Park,Café,Sushi Restaurant,Pizza Place,Bookstore,Japanese Restaurant,Bakery,Ice Cream Shop
5,Central Toronto,1,Coffee Shop,Italian Restaurant,Bakery,Café,Park,Indian Restaurant,Deli / Bodega,Restaurant,Japanese Restaurant,Bookstore
6,Central Toronto,1,Italian Restaurant,Coffee Shop,Bakery,Café,Sushi Restaurant,Japanese Restaurant,Park,Pizza Place,Bookstore,Restaurant
7,Central Toronto,1,Italian Restaurant,Coffee Shop,Pizza Place,Bakery,Indian Restaurant,Park,Japanese Restaurant,Café,Restaurant,Supermarket
8,Central Toronto,1,Italian Restaurant,Park,Sushi Restaurant,Café,Coffee Shop,Bakery,Dessert Shop,Restaurant,Indian Restaurant,Spa
9,Central Toronto,1,Italian Restaurant,Café,Coffee Shop,Park,Sushi Restaurant,Yoga Studio,Vegetarian / Vegan Restaurant,Spa,Burger Joint,Bakery
10,Downtown Toronto,1,Coffee Shop,Park,Italian Restaurant,Sushi Restaurant,Gourmet Shop,Japanese Restaurant,Spa,Indian Restaurant,Grocery Store,Breakfast Spot
23,Central Toronto,1,Coffee Shop,Italian Restaurant,Sushi Restaurant,Restaurant,Park,Japanese Restaurant,Gym,Middle Eastern Restaurant,Café,Deli / Bodega
22,Central Toronto,1,Italian Restaurant,Coffee Shop,Café,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Sushi Restaurant,Bakery,Diner,Mexican Restaurant


In [46]:
#cluster 3
toronto_merged.loc[toronto_merged["cluster Labels"]==2,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,2,Café,Bakery,Coffee Shop,Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Bookstore,Comfort Food Restaurant,Bar,Museum
24,Central Toronto,2,Italian Restaurant,Café,Coffee Shop,Vegetarian / Vegan Restaurant,French Restaurant,Museum,Grocery Store,Bakery,Restaurant,Hotel
30,Downtown Toronto,2,Café,Coffee Shop,Grocery Store,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Bar,Korean Restaurant,Comedy Club,Bakery
33,West Toronto,2,Coffee Shop,Bakery,Café,Bar,Restaurant,Furniture / Home Store,Gift Shop,Italian Restaurant,Tea Room,Cocktail Bar
32,West Toronto,2,Café,Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Cocktail Bar,Italian Restaurant,Pizza Place,Asian Restaurant,Seafood Restaurant
26,Downtown Toronto,2,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Dessert Shop,Mexican Restaurant,French Restaurant,Bakery,Pizza Place,Gaming Cafe
34,West Toronto,2,Café,Coffee Shop,Italian Restaurant,Brewery,Bar,Gastropub,Bakery,Restaurant,Pizza Place,Portuguese Restaurant
35,West Toronto,2,Café,Coffee Shop,Bakery,Park,Bar,Restaurant,Italian Restaurant,Eastern European Restaurant,Breakfast Spot,Pizza Place
36,West Toronto,2,Italian Restaurant,Bakery,Park,Coffee Shop,Bar,Café,Pizza Place,Gastropub,Scenic Lookout,Sushi Restaurant
1,East Toronto,2,Café,Greek Restaurant,Park,Vietnamese Restaurant,Bakery,Pub,Coffee Shop,Italian Restaurant,Ice Cream Shop,Cocktail Bar


In [47]:
#cluster 4
toronto_merged.loc[toronto_merged["cluster Labels"]==3,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Downtown Toronto,3,Hotel,Coffee Shop,Café,Restaurant,Seafood Restaurant,Gym,Japanese Restaurant,American Restaurant,Concert Hall,Salon / Barbershop
28,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Hotel,Seafood Restaurant,Gym,Cocktail Bar,Bakery
21,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Gym,Concert Hall,Seafood Restaurant,American Restaurant,Beer Bar
18,Downtown Toronto,3,Café,Coffee Shop,Hotel,Restaurant,Sushi Restaurant,Theater,Arts & Crafts Store,Salad Place,Furniture / Home Store,Pizza Place
16,Downtown Toronto,3,Coffee Shop,Hotel,Café,Beer Bar,Japanese Restaurant,Restaurant,Bakery,Park,Italian Restaurant,Cheese Shop
15,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Seafood Restaurant,Cosmetics Shop,Bakery,Japanese Restaurant,Italian Restaurant,Theater,Hotel
14,Downtown Toronto,3,Coffee Shop,Clothing Store,Cosmetics Shop,Tea Room,Hotel,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Bookstore,Café
13,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Theater,Bakery,Farmers Market,Park,Italian Restaurant,Hotel,Pub
12,Downtown Toronto,3,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Diner,Gym,Café,Thai Restaurant,Juice Bar
20,Downtown Toronto,3,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gym,Seafood Restaurant,Concert Hall,Japanese Restaurant,Tea Room


In [48]:
#cluster 5
toronto_merged.loc[toronto_merged["cluster Labels"]==4,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Downtown Toronto,4,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Park,Diner,Sushi Restaurant,Indian Restaurant,Gastropub,Yoga Studio
17,Downtown Toronto,4,Coffee Shop,Café,Bar,Japanese Restaurant,Italian Restaurant,Art Gallery,Thai Restaurant,Arts & Crafts Store,Sandwich Place,Bubble Tea Shop


## Obesrvations

**Most of the neighborhood fall in cluster 3 mostly with cafe and resturents etc. Cluster 1 mostly have parks and cafes. cluster 2 mostly have Italian Restaurant and cafe parks. Cluster 4 mostly have coffee shop as it is one one the business area in Toronto and Cluster 5 resturent,cafe and italian resturents.**